In [48]:
from openai import OpenAI
import pandas as pd
from dotenv import load_dotenv
import os
import json
from tqdm import tqdm
import glob

load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [6]:
def translate(input_text, target_language):
    messages = [
        {"role": "system", "content": f"""
            You are a helpful English to {target_language} translator. 
            You will be provided with an English text and you will need to translate it to {target_language}.
            Output in json format with key 'translated_text'
            """},
        {"role": "user", "content": input_text}
    ]
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        response_format={"type": "json_object"}
    )
    out = response.choices[0].message.content
    try:
        out = json.loads(out)
        return out["translated_text"]
    except:
        return out

translate("Hello, how are you?", "Chinese")

'你好，你好吗？'

In [12]:
target_language = "Japanese"

In [13]:
translation_dictionary = {
}

In [34]:
def translate_column(df, column_name):
    """
    Translates unique values in a column and adds them to translation dictionary
    
    Args:
        df: pandas DataFrame containing the column
        column_name: name of column to translate
        
    Returns:
        DataFrame with translated column
    """
    # First add any missing translations to dictionary
    for value in tqdm(df[column_name].unique(), total=len(df[column_name].unique())):
        if value not in translation_dictionary.keys():
            translation_dictionary[value] = translate(value, target_language)
    
    # Create new translated column
    translated_col = df[column_name].map(translation_dictionary)
    df[f"{column_name}"] = translated_col
    return df

# Account

In [14]:
account = pd.read_parquet("data/account.parquet")
print(account.shape)
account.head(2)

(42, 7)


,account_number,account_name,account_type_code,account_type,account_subtype_code,account_subtype,account_category
0,40001,Sales Revenue - Wholesale,REV,Revenue,INC,Income,Sales Revenue
1,40002,Sales Revenue - Direct to Consumer,REV,Revenue,INC,Income,Sales Revenue


In [21]:
# Translate specified columns
translated_account = account.copy()
for column in ["account_name", "account_type", "account_subtype", "account_category"]:
    translated_account = translate_column(translated_account, column)
translated_account.to_parquet("out/account.parquet")

100%|██████████| 13/13 [00:00<00:00, 223467.02it/s]


# Channel

In [29]:
channel = pd.read_parquet("data/channel.parquet")
print(channel.shape)
channel.head(2)

(5, 2)


,channel_number,channel_name
0,CH00001,Direct-to-Consumer (DTC)
1,CH00002,Brick-and-Mortar Retail


In [30]:
translated_channel = channel.copy()
for column in ["channel_name"]:
    translated_channel = translate_column(translated_channel, column)
translated_channel.to_parquet("out/channel.parquet")

100%|██████████| 5/5 [00:03<00:00,  1.33it/s]


# Company

In [33]:
company = pd.read_parquet("data/company.parquet")
print(company.shape)
company.head(2)

(10, 6)


,company_code,company_name,company_country,company_region,currency_code,language_code
0,AUS,EverGreen Australia Pty Ltd,Australia,Oceania,AUD,en
1,NZL,EverGreen New Zealand Ltd,New Zealand,Oceania,NZD,en


In [36]:
translated_company = company.copy()
for column in ["company_name", "company_country", "company_region"]:
    translated_company = translate_column(translated_company, column)
translated_company.to_parquet("out/company.parquet")

100%|██████████| 4/4 [00:03<00:00,  1.08it/s]


In [38]:
company_config = pd.read_json("data/company_config.json", orient="index")
company_config.head(2)

,parent_company_name,company_name,company_code,company_country,company_currency,global_currency
AUS,EverGreen Pvt Ltd,EverGreen Australia Pty Ltd,AUS,Australia,AUD,USD
BRA,EverGreen Pvt Ltd,EverGreen do Brasil Ltda,BRA,Brazil,BRL,USD


In [39]:
translated_company_config = company_config.copy()
for column in ["parent_company_name", "company_name", "company_country"]:
    translated_company_config = translate_column(translated_company_config, column)
translated_company_config.to_json("out/company_config.json", orient="index")

100%|██████████| 10/10 [00:00<00:00, 217321.45it/s]


# Config

In [41]:
config = pd.read_parquet("data/config.parquet")
print(config.shape)
config.head(2)

(1, 2)


,config_key,config_value
0,latest_closed_period,2025-P02


In [42]:
config.to_parquet("out/config.parquet")

# Cost Center

In [43]:
cost_center = pd.read_parquet("data/cost_center.parquet")
print(cost_center.shape)
cost_center.head(2)

(20, 2)


,cost_center_number,cost_center_name
0,CC1001,Manufacturing Plant A
1,CC1002,Manufacturing Plant B


In [44]:
translated_cost_center = cost_center.copy()
for column in ["cost_center_name"]:
    translated_cost_center = translate_column(translated_cost_center, column)
translated_cost_center.to_parquet("out/cost_center.parquet")

100%|██████████| 20/20 [00:13<00:00,  1.49it/s]


# Customer

In [45]:
customer = pd.read_parquet("data/customer.parquet")
print(customer.shape)
customer.head(2)

(500, 2)


,customer_number,customer_name
0,CS08001,Acme Retailers
1,CS08002,BrightMart


In [46]:
translated_customer = customer.copy()
for column in ["customer_name"]:
    translated_customer = translate_column(translated_customer, column)
translated_customer.to_parquet("out/customer.parquet")

100%|██████████| 500/500 [06:13<00:00,  1.34it/s]


# Department

In [50]:
department = pd.read_parquet("data/department.parquet")
print(department.shape)
department.head(2)

(15, 2)


,department_number,department_name
0,DP001,Research and Development
1,DP002,Product Management


In [51]:
translated_department = department.copy()
for column in ["department_name"]:
    translated_department = translate_column(translated_department, column)
translated_department.to_parquet("out/department.parquet")

100%|██████████| 15/15 [00:04<00:00,  3.27it/s]


# Division

In [54]:
division = pd.read_parquet("data/division.parquet")
print(division.shape)
division.head(2)

(6, 2)


,division_number,division_name
0,DV00001,Marketing
1,DV00002,Sales


In [55]:
translated_division = division.copy()
for column in ["division_name"]:
    translated_division = translate_column(translated_division, column)
translated_division.to_parquet("out/division.parquet")

100%|██████████| 6/6 [00:00<00:00, 10.20it/s]


# Fiscal Calendar

In [56]:
fiscal_calendar = pd.read_parquet("data/fiscal_calendar.parquet")
print(fiscal_calendar.shape)
fiscal_calendar.head(2)

(4748, 5)


,posting_date,fiscal_year,fiscal_period,fiscal_month,fiscal_quarter
0,2018-01-01,2018,P01,M01,Q1
1,2018-01-02,2018,P01,M01,Q1


In [57]:
fiscal_calendar.to_parquet("out/fiscal_calendar.parquet")

# Fiscal Period

In [58]:
fiscal_period = pd.read_parquet("data/fiscal_period.parquet")
print(fiscal_period.shape)
fiscal_period.head(2)

(156, 4)


,fiscal_year,fiscal_period,fiscal_month,fiscal_quarter
0,2018,P01,M01,Q1
1,2018,P02,M02,Q1


In [59]:
fiscal_period.to_parquet("out/fiscal_period.parquet")

# Journal

In [61]:
journal_chunk_1 = pd.read_parquet("data/journal_chunk_1.parquet")
pd.set_option('display.max_columns', None)
journal_chunk_1.head(2)

,company_code,posting_date,fiscal_year,fiscal_period,account_number,company_currency,company_amount,global_currency,global_amount,department_number,cost_center_number,profit_center_number,purchase_order_number,invoice_number,supplier_number,material_number,sales_order_number,customer_number,product_number,channel_number,division_number,transaction_id,transaction_type,transaction_document_number,transaction_document_item,transaction_description,reference_procedure
0,BRA,2025-03-23,2025,P03,40003,BRL,100006.30,USD,19048.82,DP006,CC1001,PC0004,PSP03351-2503,ISP03351-2503,SP03351,M090001,SCS08020-2503,CS08020,P060045,CH00002,DV00002,7317919591018176512,TT002,250323-713,132,,RP003
1,CHN,2022-12-16,2022,P12,61005,CNY,7720.34,USD,1069.30,DP004,CC1010,PC0005,PSP03221-2212,ISP03221-2212,SP03221,M090149,SCS08370-2212,CS08370,P060057,CH00002,DV00001,7317919591085285376,TT002,221216-048,223,,RP009


In [62]:
# Get list of journal chunk files
journal_chunks = glob.glob("data/journal_chunk_*.parquet")

# Process each chunk
for chunk_file in tqdm(journal_chunks):
    # Read chunk
    journal = pd.read_parquet(chunk_file)
    
    # nothing to translate
    
    # Save translated chunk
    output_file = chunk_file.replace("data/", "out/")
    journal.to_parquet(output_file)


100%|██████████| 15/15 [00:11<00:00,  1.27it/s]


# Material Group

In [66]:
material_group = pd.read_parquet("data/material_group.parquet")
print(material_group.shape)
material_group.head(2)

(10, 2)


,material_group_number,material_group_name
0,MG0010,Plastics
1,MG0006,Glass


In [67]:
translated_material_group = material_group.copy()
for column in ["material_group_name"]:
    translated_material_group = translate_column(translated_material_group, column)
translated_material_group.to_parquet("out/material_group.parquet")

100%|██████████| 10/10 [00:05<00:00,  1.76it/s]


# Material

In [68]:
material = pd.read_parquet("data/material.parquet")
print(material.shape)
material.head(2)

(200, 3)


,material_number,material_name,material_group_number
0,M090001,Polyethylene,MG0010
1,M090002,Polypropylene,MG0010


In [69]:
translated_material = material.copy()
for column in ["material_name"]:
    translated_material = translate_column(translated_material, column)
translated_material.to_parquet("out/material.parquet")

100%|██████████| 199/199 [01:59<00:00,  1.67it/s]


# Plan

In [71]:
plan = pd.read_parquet("data/plan.parquet")
print(plan.shape)
plan.head(2)

(1385674, 15)


,company_code,fiscal_year,fiscal_period,profit_center_number,product_number,company_currency,company_actual_amount,company_budget_amount,company_forecast_amount,company_previous_forecast_amount,global_currency,global_actual_amount,global_budget_amount,global_forecast_amount,global_previous_forecast_amount
0,AUS,2021,P01,PC0001,P060001,AUD,0.00,9727.97,0.00,0.00,USD,0.00,6442.37,0.00,0.00
1,AUS,2021,P01,PC0001,P060002,AUD,0.00,14379.47,0.00,0.00,USD,0.00,9522.83,0.00,0.00


In [72]:
plan.to_parquet("out/plan.parquet")

# Product Group

In [73]:
product_group = pd.read_parquet("data/product_group.parquet")
print(product_group.shape)
product_group.head(2)

(17, 2)


,product_group_number,product_group_name
0,PG0008,Dairy Alternatives
1,PG0013,Nuts & Spreads


In [74]:
translated_product_group = product_group.copy()
for column in ["product_group_name"]:
    translated_product_group = translate_column(translated_product_group, column)
translated_product_group.to_parquet("out/product_group.parquet")

100%|██████████| 17/17 [00:09<00:00,  1.77it/s]


# Product

In [75]:
product = pd.read_parquet("data/product.parquet")
print(product.shape)
product.head(2)

(100, 3)


,product_number,product_name,product_group_number
0,P060001,Organic Almond Milk,PG0008
1,P060002,Unsweetened Soy Milk,PG0008


In [76]:
translated_product = product.copy()
for column in ["product_name"]:
    translated_product = translate_column(translated_product, column)
translated_product.to_parquet("out/product.parquet")

100%|██████████| 100/100 [00:51<00:00,  1.96it/s]


# Profit Center

In [77]:
profit_center = pd.read_parquet("data/profit_center.parquet")
print(profit_center.shape)
profit_center.head(2)

(10, 2)


,profit_center_number,profit_center_name
0,PC0001,Beverage Division
1,PC0002,Snacks Division


In [78]:
translated_profit_center = profit_center.copy()
for column in ["profit_center_name"]:
    translated_profit_center = translate_column(translated_profit_center, column)
translated_profit_center.to_parquet("out/profit_center.parquet")

100%|██████████| 10/10 [00:07<00:00,  1.30it/s]


# Supplier

In [79]:
supplier = pd.read_parquet("data/supplier.parquet")
print(supplier.shape)
supplier.head(2)

(500, 2)


,supplier_number,supplier_name
0,SP03001,Acme Packaging Solutions
1,SP03002,GreenLeaf Naturals


In [80]:
translated_supplier = supplier.copy()
for column in ["supplier_name"]:
    translated_supplier = translate_column(translated_supplier, column)
translated_supplier.to_parquet("out/supplier.parquet")

100%|██████████| 499/499 [06:11<00:00,  1.34it/s]
